<a href="https://colab.research.google.com/github/Soma-codeAI/Google_ADK_Examples/blob/main/notebooks/StockPrice_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install yfinance

In [ ]:
from google.colab import auth

auth.authenticate_user(project_id="<Project_ID>")

In [ ]:
import vertexai
from vertexai import agent_engines

vertexai.init(
    project="<Project_ID>",
    location="us-east4",
    staging_bucket="<Gcloud Storage Bucket>",
)

In [ ]:
# Set up guardrails for the model
from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

In [ ]:
# Content Generation parameters, This is where you get your model to be creative
from google.genai import types


generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.28,
   max_output_tokens=1000,
   top_p=0.95,
)

In [ ]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=stock_price_agent,
    enable_tracing=True,
)

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import datetime
import requests
from zoneinfo import ZoneInfo
from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp
from vertexai.generative_models import FunctionDeclaration, Tool, Part
from google.adk.tools import FunctionTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

import yfinance as yf


model1 = vertexai.generative_models.GenerativeModel(model_name="gemini-2.0-flash")
GOOGLE_GENAI_USE_VERTEXAI='TRUE'

APP_NAME = "stock_app"
USER_ID = "1234"
SESSION_ID = "session1234"

def get_stock_price(symbol: str):
    """
    Retrieves the current stock price for a given symbol.

    Args:
        symbol (str): The stock symbol (e.g., "AAPL", "GOOG").

    Returns:
        float: The current stock price, or None if an error occurs.
    """
    try:
        stock = yf.Ticker(symbol)
        historical_data = stock.history(period="1d")
        if not historical_data.empty:
            current_price = historical_data['Close'].iloc[-1]
            return current_price
        else:
            return None
    except Exception as e:
        print(f"Error retrieving stock price for {symbol}: {e}")
        return None

stock_price_agent = Agent(
    name='stock_agent',
    model = "gemini-2.0-flash",                 # Required.
    generate_content_config=generate_content_config,  # Optional.
    instruction= 'You are an agent who retrieves stock prices. If a ticker symbol is provided, fetch the current price. If only a company name is given, first perform a Google search to find the correct ticker symbol before retrieving the stock price. If the provided ticker symbol is invalid or data cannot be retrieved, inform the user that the stock price could not be found.',
    description='This agent specializes in retrieving real-time stock prices. Given a stock ticker symbol (e.g., AAPL, GOOG, MSFT) or the stock name, use the tools and reliable data sources to provide the most up-to-date price.',
    tools=[get_stock_price]# You can add Python functions directly to the tools list; they will be automatically wrapped as FunctionTools.
    )

app = AdkApp(agent=stock_price_agent)

In [ ]:
app.list_sessions(user_id="u_123")
session = app.create_session(user_id="u_123")

for event in app.stream_query(
      user_id="u_123",
      session_id=session.id,
      message="stock price of GOOG",
    ):
      print(event)